In [85]:
import pandas as pd
df = pd.read_csv("C:/Users/z1xtr/Downloads/Naive_bayes/Naive_bayes/tweets.csv")

In [86]:
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [87]:
tweet = df.tweet

In [88]:
label = df.label

In [89]:
tweet.head()

0    #fingerprint #Pregnancy Test https://goo.gl/h1...
1    Finally a transparant silicon case ^^ Thanks t...
2    We love this! Would you go? #talk #makememorie...
3    I'm wired I know I'm George I was made that wa...
4    What amazing service! Apple won't even talk to...
Name: tweet, dtype: object

In [90]:
label.head()

0    0
1    0
2    0
3    0
4    1
Name: label, dtype: int64

In [91]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [92]:
import re
pattern = '[{}]'.format(re.escape(string.punctuation))
pattern

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~]'

In [93]:
import nltk
stw = nltk.corpus.stopwords.words('english')
print(stw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Defining a function for text pre-processing

In [94]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

# converting the text into all lower cases

def text_lower(text):
    tlower = pd.Series([sents.lower() for sents in text])
    return tlower

# word tokenization

def word_tkns(text):
    tkns = pd.Series([nltk.tokenize.word_tokenize(sents) for sents in text])
    return tkns

# removing punctuation

def punct_removal(text):
    punct_pattern = '[{}]'.format(re.escape(string.punctuation))
    regex_pattern = re.compile(punct_pattern)
    clean_sents1= pd.Series([list(filter(None, [regex_pattern.sub('',  x) for x in sents])) for sents in text])
    return clean_sents1

# stopword removal
# remember to not remove negations in the case of 

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    clean_sents2 = pd.Series([[x for x in sents if x not in stopwords] for sents in text])
    return clean_sents2


## correcting words like spelling mistakes, repeated characters, etc. 

from nltk.corpus import wordnet as wn
def remove_repeated_character(text):
    
    # pattern that occur twice among other characters
    pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    replacement = r'\1\2\3'
    
    def replace(tkn):
        
        # check semantically correct word if not replace and check again
        if wn.synsets(tkn):
            return tkn
        
        tkn_c = pattern.sub(replacement, tkn)
        
        # recursive call
        return replace(tkn_c) if tkn_c != tkn else tkn_c
    
     # correct each token 
    token_c = pd.Series([[replace(tn) for tn in sents] for sents in text])
    return token_c

## PoS tagging

def pos_tag_text(text):
    
    tkn_tagged = pd.Series([nltk.pos_tag(sents, tagset= 'universal') for sents in text])
    
    def penn_to_wn(ptag):
        
        if ptag.startswith('N'):
            return wn.NOUN
        if ptag.startswith('V'):
            return wn.VERB
        if ptag.startswith('J'):
            return wn.ADJ
        if ptag.startswith('R'):
            return wn.ADV
        else:
            return None 
    
    tkn_tagged_wn = pd.Series([[(token, penn_to_wn(tag)) for token, tag in sents] for sents in tkn_tagged])
    return tkn_tagged_wn
                             
## Lemmatization using WordNetLemmatizer

def wnl(text):
    wnlemmatizer = WordNetLemmatizer()
    stemmed_text = pd.Series([[wnlemmatizer.lemmatize(words, tag) if tag else wnlemmatizer.lemmatize(words) for words, tag in sents ] for sents in text])
    return stemmed_text                          

# Converting earch element in the series from a list to string

def list_to_string(text):
    str_series = pd.Series([' '.join(sents) for sents in text])
    return str_series
    
# Defining a function for text pre-processing

def preprocessed_text(text):
    
    tl = text_lower(text)
    wtkns = word_tkns(tl)
    punc_removal = punct_removal(wtkns)
    remove_stw = remove_stopwords(punc_removal)
    removing_extra_characters = remove_repeated_character(remove_stw) 
    wn_tagged = pos_tag_text(removing_extra_characters)
    lemmatized_text = wnl(wn_tagged)
    preprocessed_text= list_to_string(lemmatized_text)
    return preprocessed_text



## Testing the function preprocessed_text(text) on tweets

In [95]:
tweet_c = preprocessed_text(tweet)
print(tweet_c[0:5])

0    fingerprint pregnancy test http goglh1mfqv and...
1    finally transparant silicon case thanks uncle ...
2    love would go talk makememories unplug relax i...
3    wire know george make way iphone cute daventry...
4    amazing service apple wo nt even talk question...
dtype: object


# FEATURE EXTRACTION USING TF-IDF MODEL

### Creating train and test partition

In [96]:
from sklearn.model_selection import train_test_split
tweet_train, tweet_test, label_train, label_test = train_test_split(tweet_c, label, test_size= 0.3, random_state=42, shuffle=True)

In [97]:
tweet_train.head()

4015    love apple watch apple watch 42m excited chris...
6898    girl demi lavato lavatic cute sexy fine music ...
3779    new smartphone sony xperia new smartphone dj l...
1611    2 gift birthday gift metome usa nike nikeair i...
4204    backend vocation newyear trip thailand khaoko ...
dtype: object

In [98]:
label_train.head()

4015    0
6898    0
3779    0
1611    0
4204    0
Name: label, dtype: int64

# Extracting features using TfidfVectorizer

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

# creating an instance of TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(norm='l2', smooth_idf=True, use_idf=True)

# using fit_transform method to extract the features from tweet-train
features_train_t = tfidfvectorizer.fit_transform(tweet_train)
features_train_tm = features_train_t.toarray()

# tranforming the tweet-test
features_test_t = tfidfvectorizer.transform(tweet_test)
features_test_tm = features_test_t.toarray()

# get feature names
feature_names_t = tfidfvectorizer.get_feature_names_out()
print(feature_names_t[:50])

['01' '0101' '0101am' '0101amay' '010315' '015' '01634' '02' '0230pm' '03'
 '0301am' '0301amjune' '030am' '0340pm' '04' '040am' '040pm' '0430pm'
 '0490love' '05' '051' '0530am' '06' '0630am' '069' '07' '070' '0701'
 '0701am' '0701amapril' '070am' '070pm' '0730pm' '07yilmaz242' '08'
 '0801amay' '0801pm' '0801pmapril' '0801pmjune' '080am' '080pm'
 '0818482028' '0830am' '0830pm' '09' '090pm' '0930am' '0g' '0x' '0ä1ö']


In [100]:
# creating dataframe
df_train_t = pd.DataFrame(data=features_train_tm, columns= feature_names_t)
df_test_t = pd.DataFrame(data=features_test_tm, columns= feature_names_t)

print(df_train_t.iloc[0:5, 0:10])
print(df_test_t.iloc[0:5, 0:10])

    01  0101  0101am  0101amay  010315  015  01634   02  0230pm   03
0  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
1  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
2  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
3  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
4  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
    01  0101  0101am  0101amay  010315  015  01634   02  0230pm   03
0  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
1  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
2  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
3  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0
4  0.0   0.0     0.0       0.0     0.0  0.0    0.0  0.0     0.0  0.0


## Implement Multinomial Naive Bayes

In [101]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# create an instance of MultinomialNB()
multinomialnb = MultinomialNB()

# train the model
multinomialnb.fit(features_train_t, label_train)
# make_predictions
predictions_t = multinomialnb.predict(features_test_t)

# check accuracy
as_t = accuracy_score(label_test, predictions_t)
print('accuracy score:', as_t)

accuracy score: 0.8303872053872053


# classification matrix

In [102]:
#classification matrix
from sklearn import metrics
cm_multiNB = pd.DataFrame(data= metrics.confusion_matrix(label_test, predictions_t))
cm_multiNB

,0,1
0,1691,44
1,359,282


In [103]:
# total 359 wrong prediction for class 1
df_predictions_check = pd.DataFrame({'tweet' : tweet_test, 'predicted_label' : predictions_t, 'label_t' : label_test})
df_predictions_check[df_predictions_check['label_t'] == 1][df_predictions_check['predicted_label'] == 0]

C:\Users\z1xtr\AppData\Local\Temp\ipykernel_38088\1918579730.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_predictions_check[df_predictions_check['label_t'] == 1][df_predictions_check['predicted_label'] == 0]


,tweet,predicted_label,label_t
7539,skulcandy product brutal 1 headphone always st...,0,1
2464,ragin fbtwiterinsta aps phone aw card detail r...,0,1
6597,mi iphone5 connect itunes geniusbar appointmen...,0,1
554,go apple store fourth time wednesday getingold,0,1
2266,nice line aps iphone aps fuckyou funny naughty...,0,1
...,...,...,...
5404,apple marketing best planet suck sucker apple ...,0,1
2925,apple aplemusic overly complicate unreliable s...,0,1
7176,dreamt emojis tripy af nt even iphone becausef...,0,1
3856,16gb iphone 5 sell 649 material worth 168 acco...,0,1


In [104]:
df.iloc[7539]

id                                                    7540
label                                                    1
tweet    @skullcandy your product is brutal, 1 headphon...
Name: 7539, dtype: object

In [105]:
df.iloc[554]

id                                                     555
label                                                    1
tweet    Have to go to the Apple Store for the fourth t...
Name: 554, dtype: object

## Implement Bernoulli Naive Bayes

In [106]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
bernoullinb2 = BernoulliNB()
bernoullinb2.fit(features_train_t, label_train)
predictions_bnb2 = bernoullinb2.predict(features_test_t)

as_bnb2 = accuracy_score(label_test, predictions_bnb2)
print('accuracy score:', as_bnb2)

accuracy score: 0.8754208754208754


## classification matrix

In [107]:
pd.DataFrame(data= metrics.confusion_matrix(label_test, predictions_bnb2))

,0,1
0,1652,83
1,213,428


# Support Vector Machine (SVM) classifier

In [108]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
svm1 = SGDClassifier()
svm1.fit(features_train_t, label_train)
predictions_svm1 = svm1.predict(features_test_t)

print('accuracy score:', metrics.accuracy_score(label_test, predictions_svm1), '\nprecision score:', metrics.precision_score(label_test, predictions_svm1))

accuracy score: 0.8813131313131313 
precision score: 0.7774343122102009


## classification matrix

In [109]:
pd.DataFrame(data= metrics.confusion_matrix(label_test, predictions_svm1))

,0,1
0,1591,144
1,138,503
